<a href="https://colab.research.google.com/github/dipta13/CNN-PreTrainedModel-Inception/blob/main/CobaTubesCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
from google.colab import drive

In [93]:
cd /content/drive/MyDrive/DATASET

/content/drive/MyDrive/DATASET


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np

import os

In [12]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size 
INIT_LR= 1e-4
EPOCHS = 5
BS = 32

In [13]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images("/content/drive/MyDrive/DATASET/JERAWAT"))
data = []
labels = []

[INFO] loading images...


In [14]:
# loop over the image paths
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# load the input image (224x224) and preprocess it
	image = load_img(imagePath,target_size=(224,224))
	print(image)
	image = img_to_array(image)
	image = preprocess_input(image)

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)

<PIL.Image.Image image mode=RGB size=224x224 at 0x7F1C27333690>
<PIL.Image.Image image mode=RGB size=224x224 at 0x7F1C27333690>
<PIL.Image.Image image mode=RGB size=224x224 at 0x7F1C27333690>
<PIL.Image.Image image mode=RGB size=224x224 at 0x7F1C27333690>
<PIL.Image.Image image mode=RGB size=224x224 at 0x7F1C27333690>
<PIL.Image.Image image mode=RGB size=224x224 at 0x7F1C226DBE90>
<PIL.Image.Image image mode=RGB size=224x224 at 0x7F1C226DB850>
<PIL.Image.Image image mode=RGB size=224x224 at 0x7F1C226DBE50>
<PIL.Image.Image image mode=RGB size=224x224 at 0x7F1C226DBE90>
<PIL.Image.Image image mode=RGB size=224x224 at 0x7F1C27333690>
<PIL.Image.Image image mode=RGB size=224x224 at 0x7F1C226DBE90>
<PIL.Image.Image image mode=RGB size=224x224 at 0x7F1C226DBB50>
<PIL.Image.Image image mode=RGB size=224x224 at 0x7F1C27333690>
<PIL.Image.Image image mode=RGB size=224x224 at 0x7F1C226DB850>
<PIL.Image.Image image mode=RGB size=224x224 at 0x7F1C27333690>
<PIL.Image.Image image mode=RGB size=224

In [15]:
print(labels)

['JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT', 'JERAWAT'

In [16]:
# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [17]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)

In [18]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")


In [19]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

9420800/9406464 [==============================] - 0s 0us/step


In [20]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

In [21]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])


[INFO] compiling model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
